In [1]:
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import skimage.io as io
import time
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Conv2D,MaxPooling2D,Flatten
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint
from tensorflow.keras.models import load_model
from sklearn.decomposition import PCA

In [1]:
#Processing of the label set data into the form used in the model
def labelprocessing(label_trainpath,label_testpath):
    #Valid data splitting of read-in training csv, table header renaming
    raw_data_train=pd.read_csv(label_trainpath,names=["mixed_information_train"])
    data_unmixed_train = raw_data_train["mixed_information_train"].str.split("\t",expand=True)
    data_unmixed_train.columns=['index','eye_color','face_shape','file_name']
    data_unmixed_train.drop(columns=['index','eye_color','file_name'],inplace=True)
    data_unmixed_train.drop(index=0,inplace=True)
    #Perform the same processing on the read-in test set results
    raw_data_test=pd.read_csv(label_testpath,names=["mixed_information_test"])
    data_unmixed_test = raw_data_test["mixed_information_test"].str.split("\t",expand=True)
    data_unmixed_test.columns=['index','eye_color','face_shape','file_name']
    data_unmixed_test.drop(columns=['index','eye_color','file_name'],inplace=True)
    data_unmixed_test.drop(index=0,inplace=True)
    #Store the results of training and testing as arrays
    YTrain=data_unmixed_train
    YTest=data_unmixed_test
    YTrain=YTrain.values
    YTest=YTest.values
    YTrain=np.array([int(y) for y in YTrain])
    YTest=np.array([int(y) for y in YTest])
    YTrain=YTrain.reshape(len(YTrain),1)
    YTest=YTest.reshape(len(YTest),1)
    
    return YTrain,YTest

#Downscaling high-dimensional images
def change_shape(img):
    [NUM,row,col,dimension]=img.shape
    img=img.reshape(NUM,row*col*dimension)
    return img

#Processing of the data set data into the form used in the model,includes reading, image dimensionality reduction and normalisation, processing of input data
def datapreprocessing(data_train_path,data_test_path):
    XTrain_tmp=[]
    XTest_tmp=[]
    XTrain=[]
    XTest=[]
    coll_train = io.ImageCollection(data_train_path)
    coll_test = io.ImageCollection(data_test_path)
    for img in coll_train:
        intern_train=cv2.resize(img, (120,120))
        XTrain_tmp.append(intern_train)
    XTrain_tmp=np.array(XTrain_tmp)/255
    for img in coll_test:
        intern_test=cv2.resize(img, (120,120))
        XTest_tmp.append(intern_test)
    XTest_tmp=np.array(XTest_tmp)/255
    for (i,value) in enumerate(XTest_tmp):
        XTest.append(value.flatten())
    for (i,value) in enumerate(XTrain_tmp):
        XTrain.append(value.flatten())
    XTrain=np.array(XTrain)
    XTest=np.array(XTest)
    XTrain=XTrain.reshape(len(XTrain),120,120,4)
    XTest=XTest.reshape(len(XTest),120,120,4)
    return XTrain,XTest

#Performing PCA on the input data and building neural networks
def predict(XTrain,YTrain):
    import time
    XTrain=change_shape(XTrain)
    pca=PCA(n_components=200,svd_solver='randomized',whiten=True).fit(XTrain)
    XTrain = pca.transform(XTrain)
    start=time.perf_counter()
    fully_connected_layer_num=[3]
    unit_count=[200]
    for fully_connected_layer in fully_connected_layer_num:
        for unit in unit_count:
            model=Sequential()
            for i in range(fully_connected_layer):
                model.add(Dense(unit,activation='relu'))
            #output layer
            model.add(Dense(5,activation='softmax'))
            #compile
            model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
            #fit
            model.fit(XTrain,YTrain,batch_size=64,epochs=5)
    end=time.perf_counter()
    time=end-start
    return model,pca,time

#Return Forecast Results
def acc_Test(model,XTest,YTest):
    Yres=[]
    res=model.predict(XTest)
    for i in range(len(res)):
        Yres.append(np.argmax(res[i]))
    Yres=np.array(Yres)
    acc=np.mean(Yres==YTest.flatten())
    return acc

def main(label_trainpath,label_testpath,data_train_path,data_test_path):
    YTrain,YTest=labelprocessing(label_trainpath,label_testpath)
    XTrain,XTest=datapreprocessing(data_train_path,data_test_path)
    model,pca,time=predict(XTrain,YTrain)
    XTest=change_shape(XTest)
    XTest = pca.transform(XTest)
    acc=acc_Test(model,XTest,YTest)
    return acc

In [2]:
#Data paths
label_trainpath=r".\Datasets\cartoon_set\labels.csv"
label_testpath=r".\Datasets\cartoon_set_test\labels.csv"
data_train_path=r".\Datasets\cartoon_set\img\*.png"
data_test_path= r".\Datasets\cartoon_set_test\img\*.png"

In [4]:
acc=main(label_trainpath,label_testpath,data_train_path,data_test_path)
print("TaskB1: Accuracy of Fully Connected Neural Network is {}".format(acc))

Epoch 1/5
157/157 [==============================] - 1s 2ms/step - loss: 0.3752 - accuracy: 0.8721
Epoch 2/5
157/157 [==============================] - 0s 2ms/step - loss: 0.0188 - accuracy: 0.9957
Epoch 3/5
157/157 [==============================] - 0s 2ms/step - loss: 0.0056 - accuracy: 0.9988
Epoch 4/5
157/157 [==============================] - 0s 2ms/step - loss: 0.0076 - accuracy: 0.9977
Epoch 5/5
79/79 [==============================] - 0s 1ms/step
